## PyTorch Datasets

In [ ]:
from torchvision import transforms
from torchvision.transforms import InterpolationMode
import sys
import os

sys.path.append(os.path.abspath(".."))
from src.dataset import ImageDataset

train_transform = transforms.Compose([
    transforms.Resize((272, 272), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomResizedCrop(
        (256, 256),
        scale=(0.85, 1.0),
        ratio=(0.9, 1.1),
        interpolation=InterpolationMode.BILINEAR,
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.12, contrast=0.12, saturation=0.08),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

eval_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

root_train = os.path.join('..', 'data', 'ImageNetSubset')
root_test = os.path.join('..', 'data', 'collected_dataset')

dataset_train = ImageDataset(root=root_train, transform=train_transform, train=True)
dataset_eval = ImageDataset(root=root_train, transform=eval_transform, train=True)
dataset_test = ImageDataset(root=root_test, transform=eval_transform, train=False)


In [15]:
#check if paths are correct in different machines / operating systems

print("Train path:", os.path.abspath(root_train))
print("Exists?", os.path.exists(root_train))

print("Test path:", os.path.abspath(root_test))
print("Exists?", os.path.exists(root_test))


Train path: c:\Users\klugm\Documents\Studium\BA_AI\5. Semester\xai-proj\xai_proj_b_group_404\data\ImageNetSubset
Exists? True
Test path: c:\Users\klugm\Documents\Studium\BA_AI\5. Semester\xai-proj\xai_proj_b_group_404\data\collected_dataset
Exists? True


## PyTorch Dataloaders

In [ ]:
from torch import Generator
from torch.utils.data import random_split, DataLoader, Subset

n = len(dataset_train)
train_size = int(0.8 * n)
val_size = n - train_size

train_idx, val_idx = random_split(
    range(n),
    [train_size, val_size],
    generator=Generator().manual_seed(42),
)

train_dataset = Subset(dataset_train,  train_idx.indices)   # augmented
val_dataset   = Subset(dataset_eval, val_idx.indices)     # clean

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
testloader = DataLoader(dataset_test, batch_size=64, shuffle=False, num_workers=4)

## Import Model

In [17]:
from src.models import SimpleCNN, LargerNet

net = SimpleCNN()

## Training Loop

In [ ]:
# vorher per pip3 tensorflow installieren

In [18]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [21]:
from torch.utils.tensorboard import SummaryWriter
from torch import nn, optim 
from sklearn.metrics import f1_score, roc_auc_score
import torch
import time

net = SimpleCNN()
early_stopper = EarlyStopper(patience=5, min_delta=0.1)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# Device selection: MPS (Apple Silicon) > CUDA > CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using device:", device)
net.to(device)

writer = SummaryWriter(log_dir=f'./runs/LargerNet/exp_{int(time.time())}')

num_epochs = 5

for epoch in range(num_epochs):
    # --- Training ---
    net.train()
    running_loss_sum = 0.0
    train_samples = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        bs = inputs.size(0)
        running_loss_sum += loss.item() * bs
        train_samples += bs

    # --- Validation ---
    net.eval()
    val_loss_sum = 0.0
    val_samples = 0
    correct = 0
    val_predictions = []
    val_true_labels = []
    val_probs = []

    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            probs = torch.softmax(outputs, dim=1)

            bs = inputs.size(0)
            val_loss_sum += loss.item() * bs
            val_samples += bs

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()

            val_predictions.append(predicted.cpu())
            val_true_labels.append(labels.cpu())
            val_probs.append(probs.cpu())

    #Calculate Loss
    avg_train_loss = running_loss_sum / train_samples
    avg_val_loss = val_loss_sum / val_samples if val_samples > 0 else 0.0

    #Calculate Accuracy
    val_acc = correct / val_samples if val_samples > 0 else 0.0
    
    #Calculate F1 Score
    y_true = torch.cat(val_true_labels).numpy()
    y_pred = torch.cat(val_predictions).numpy()
    f1_weighted = f1_score(y_true, y_pred, average='weighted')

    #Calculate AUC-ROC
    y_probs = torch.cat(val_probs).numpy()
    try:
        roc_auc = roc_auc_score(y_true, y_probs, multi_class='ovr', average='weighted')
    except ValueError:
        roc_auc = float('nan')  # if not all classes are present in y_true

    # --- TensorBoard Logging ---
    writer.add_scalar('Loss/train_epoch', avg_train_loss, epoch)
    writer.add_scalar('Loss/val_epoch', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/val_epoch', val_acc, epoch)
    writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)
    writer.add_scalar('F1_Score/val_epoch', f1_weighted, epoch)
    writer.add_scalar('AUC_ROC/val_epoch', roc_auc, epoch)
 

    print(
        f'Epoch {epoch+1}/{num_epochs} '
        f'- train_loss: {avg_train_loss:.4f} '
        f' val_loss: {avg_val_loss:.4f} '
        f' val_acc: {val_acc:.4f}'
        f' LR: {optimizer.param_groups[0]["lr"]:.6f}'
        f' F1_score: {f1_weighted:.4f}'
        f' AUC_ROC: {roc_auc:.4f}'
    )

    scheduler.step(avg_val_loss)

    if early_stopper.early_stop(avg_val_loss):
        print('Stopped\nMin Validation Loss:', early_stopper.min_validation_loss)
        break


print('Finished Training')
writer.close()

Using device: cpu
Epoch 1/5 - train_loss: 2.1522  val_loss: 2.0421  val_acc: 0.2619 LR: 0.001000 F1_score: 0.2336 AUC_ROC: 0.7137
Epoch 2/5 - train_loss: 1.9543  val_loss: 1.9525  val_acc: 0.3070 LR: 0.001000 F1_score: 0.2831 AUC_ROC: 0.7581


KeyboardInterrupt: 

## Model Testen

In [ ]:
correct = 0
total = 0

all_predictions = torch.Tensor()
all_labels = torch.Tensor()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predictions = torch.cat((all_predictions, predicted.cpu()))
        all_labels = torch.cat((all_labels, labels.cpu()))

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

## Ergebnisse Speichern

In [ ]:
# ACHTUNG: Dateinamen anpassen! sonst wird altes Modell überschrieben
PATH = 'image_net_2511xx_2.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
!tensorboard --logdir='c:\Users\elias\Downloads\events.out.tfevents.1764768203.db0cf8b52daa.47.3'